<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/allosaurus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Allosaurus を利用した発音記号の特定

論文：https://arxiv.org/abs/2002.11800

ソース：https://github.com/xinjli/allosaurus

In [ ]:
pip install pydub

### Allosaurus のインストール

In [ ]:
pip install allosaurus

### 録音用関数 (record)

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def record_js(filename='record.mp3'):
  js = Javascript('''
    async function record() {
      let rec;
      let chanks;


      // HTML組み立て
      const div = document.createElement('div');
      const startRecord = document.createElement('button');
      startRecord.textContent = 'Rec';
      div.appendChild(startRecord);

      const stopRecord = document.createElement('button');
      stopRecord.textContent = 'Stop';
      stopRecord.style.display = 'none'
      div.appendChild(stopRecord);

      const audio = document.createElement('audio');
      div.appendChild(audio);

      document.body.appendChild(div);

      // Audioが有効になったら
      function handlerFunction(stream,resolve) {
          rec = new MediaRecorder(stream);
          // 録音が完了したら
          rec.ondataavailable = e => {
              chanks.push(e.data);
              if (rec.state == "inactive") {
                  let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
                  audio.src = URL.createObjectURL(blob);
                  audio.controls = true;
                  audio.autoplay = true;
                  resolve();
              }
          }
      }

      startRecord.onclick = e => {
          startRecord.style.display = 'none'
          stopRecord.style.display = 'block'
          chanks = [];
          rec.start();
      }

      stopRecord.onclick = e => {
        startRecord.style.display = 'block'
        stopRecord.style.display = 'none'
        rec.stop();
      }

      function blobToBase64(blob) {
        return new Promise((resolve, _) => {
          const reader = new FileReader();
          reader.onloadend = () => resolve(reader.result);
          reader.readAsDataURL(blob);
        });
      }

      await new Promise((resolve) => {
        navigator.mediaDevices.getUserMedia({ audio: true })
            .then(stream => { handlerFunction(stream,resolve) })
      });
      let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
      return await blobToBase64(blob);
    }
    ''')
  display(js)
  data = eval_js('record()')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

### 録音

In [ ]:
import pydub
import IPython.display

record_js()
!yes|ffmpeg -i record.mp3 -ac 1 -f wav record.wav > /dev/null 2>&1
IPython.display.Audio("record.wav", autoplay=True)

### 音声 --> 発音記号

In [ ]:
from allosaurus.app import read_recognizer

# load your model
model = read_recognizer()

# run inference -> æ l u s ɔ ɹ s
model.recognize('record.wav')